In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPool2D, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models, losses
import numpy as np 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
x_train = x_train.astype('float32') / 255.0 
x_test = x_test.astype('float32') / 255.0 

In [ ]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
x_train.shape[1:]

(32, 32, 3)

In [ ]:
model = Sequential()
model.add(layers.experimental.preprocessing.Resizing(227, 227, interpolation="bilinear", input_shape=x_train.shape[1:]))
model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=2))
model.add(BatchNormalization())

model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), strides=1, activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D((3,3), strides=2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_8 (Resizing)        (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 57, 57, 96)        34944     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 28, 28, 256)       614656    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 14, 14, 256)      

In [ ]:
model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, 
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Epoch 1/50
391/391 [==============================] - 61s 146ms/step - loss: 1.8079 - accuracy: 0.4025 - val_loss: 1.5928 - val_accuracy: 0.4405
Epoch 2/50
391/391 [==============================] - 56s 143ms/step - loss: 1.2285 - accuracy: 0.5644 - val_loss: 1.1993 - val_accuracy: 0.5666
Epoch 3/50
391/391 [==============================] - 57s 145ms/step - loss: 1.0247 - accuracy: 0.6435 - val_loss: 1.3138 - val_accuracy: 0.5728
Epoch 4/50
391/391 [==============================] - 57s 147ms/step - loss: 0.8745 - accuracy: 0.6966 - val_loss: 2.4099 - val_accuracy: 0.2241
Epoch 5/50
391/391 [==============================] - 58s 147ms/step - loss: 0.7887 - accuracy: 0.7273 - val_loss: 0.9296 - val_accuracy: 0.6752
Epoch 6/50
391/391 [==============================] - 57s 147ms/step - loss: 0.7045 - accuracy: 0.7575 - val_loss: 1.1303 - val_accuracy: 0.6151
Epoch 7/50
391/391 [==============================] - 57s 147ms/step - loss: 0.6374 - accuracy: 0.7833 - val_loss: 0.9113 - val_ac

(50000, 32, 32, 3)